# Predict climate from berry fungal communities

> using `ritme` (https://github.com/adamovanja/ritme)

note. use the **ritme_model** environment! 


In [31]:
from ritme.find_best_model_config import (
    _load_experiment_config,
    _load_phylogeny,
    _load_taxonomy,
    find_best_model_config,
)
from ritme.split_train_test import _load_data, split_train_test
from ritme.evaluate_tuned_models import evaluate_tuned_models

import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/climate-berries'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/climate-berries


In [33]:
# set experiment configuration path - this is downloaded from the github repo
# here we specify the column to be stratified by and the target! 
model_config_path = "/home/lfloerl/microterroir/Microbiome/Other_scripts/config/r_local_linreg_py.json"

# define path to feature table, metadata, and taxonomy (no phylogeny)
path_to_ft = "climate_filtered_table.qza"
path_to_md = "/home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_Climate.tsv"
path_to_tax = "/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/taxonomy.qza"

# define train size
train_size = 0.8

In [34]:
config = _load_experiment_config(model_config_path)

In [35]:
md_df = pd.read_csv(path_to_md, sep='\t')
md_df.head()

,id,SAMPLE_NAME,sample_type,Plot_ID,COLLECTION_DATE,Year,Year_Cat,Plot,Altitude,Average_slope,...,maximum_rh,minimum_rh,cv_rh,GDD,average_temperature,median_temperature,maximum_temperature,minimum_temperature,accumulated_temperature,cv_temperature
0,364526_290-LP3-ITS-0866,Lavaux_2021-08-31_bark_Plot4,bark,Lavaux_Plot_4,2021-08-31 00:00:00 +0200,2021,Year2021,4,450,30,...,99.036583,33.078261,16.567322,1325.721843,18.154261,18.650958,26.689636,8.401750,2940.990343,22.587667
1,364526_289-LP3-ITS-0865,Lavaux_2021-08-31_bark_Plot5,bark,Lavaux_Plot_5,2021-08-31 00:00:00 +0200,2021,Year2021,5,400,30,...,95.964333,32.830458,16.331714,1379.347141,18.494554,19.047366,26.351409,8.816375,2996.117777,22.104867
2,364526_285-LP3-ITS-0861,Lavaux_2021-08-31_bark_Plot9,bark,Lavaux_Plot_9,2021-08-31 00:00:00 +0200,2021,Year2021,9,520,35,...,98.181542,32.794125,16.570451,1286.254021,17.902585,18.256629,25.850773,8.052083,2900.218734,22.831881
3,364526_287-LP3-ITS-0863,Lavaux_2021-08-31_bark_Plot11,bark,Lavaux_Plot_11,2021-08-31 00:00:00 +0200,2021,Year2021,11,490,20,...,96.437167,32.357792,16.118351,1297.271087,17.964399,18.541438,26.775286,7.966542,2910.232671,23.246939
4,364526_282-LP3-ITS-0858,Lavaux_2021-08-31_bark_Plot12,bark,Lavaux_Plot_12,2021-08-31 00:00:00 +0200,2021,Year2021,12,520,15,...,97.113875,32.104636,16.661423,1286.822875,17.897069,18.244984,27.509143,7.779875,2899.325119,23.615955


## Read & split data


In [36]:
md, ft = _load_data(path_to_md, path_to_ft)
print(md.shape, ft.shape)

(595, 35) (326, 6091)


In [37]:
train_val, test = split_train_test(
    md,
    ft,
    stratify_by_column=config["stratify_by_column"],
    feature_prefix=config["feature_prefix"],
    train_size=train_size,
    seed=config["seed_data"],
)

Train: (221, 6126), Test: (105, 6126)


/scratch/lfloerl/.condaenvs/ritme_model/lib/python3.10/site-packages/ritme/split_train_test.py:135: UserWarning: Provided feature table contains absolute instead of relative abundances. Hence, converting it to relative abundances...
  warnings.warn(


## Find best model config


In [39]:
tax = _load_taxonomy(path_to_tax)

best_model_dict, path_to_exp = find_best_model_config(
    config, train_val, tax, path_store_model_logs="ritme_refact_logs_RF_LR")

2025-02-11 11:58:58,457	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 0.618 s, which may be a performance bottleneck.
2025-02-11 11:58:58,459	WARNING util.py:201 -- The `process_trial_result` operation took 0.620 s, which may be a performance bottleneck.
2025-02-11 11:58:58,460	WARNING util.py:201 -- Processing trial results took 0.621 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2025-02-11 11:58:58,460	WARNING util.py:201 -- The `process_trial_result` operation took 0.622 s, which may be a performance bottleneck.
2025-02-11 11:59:04,713	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 0.619 s, which may be a performance bottleneck.
2025-02-11 11:59:04,715	WARNING util.py:201 -- The `process_trial_result` operation took 0.620 s, which may be a performance bottleneck.
2025-02-11 11:59:04,715	WARNING util.py:201 -- Processing trial results took 0.621 s, which may be a performance bott

## Evaluate best models


In [22]:
metrics = evaluate_tuned_models(best_model_dict, config, train_val, test)
metrics

,rmse_train,r2_train,rmse_test,r2_test
linreg,0.766429,-0.016003,0.648017,-0.464744
